In [7]:
import sqlite3
import pandas as pd
from glob import glob
import re

In [8]:
def extract_number_from_path(path):
    match = re.search(r'(\d+)\.dcm$', path)
    if match:
        return int(match.group(1))
    return 0

def get_data_for_3d_volumes(data,train_data_cat, path, number_idx):
    
    data_to_merge = data[["patient_id", "series_id"]]
    patient_category = train_data_cat[["patient_id", "any_injury"]]
    
    merged_df = data_to_merge.merge(patient_category, on='patient_id', how='left')
    
    shuffled_data = merged_df.sample(frac=1, random_state=42)
    shuffled_indexes = shuffled_data.index[:number_idx]
    selected_rows = shuffled_data.loc[shuffled_indexes]
    data_to_merge_processed = selected_rows.reset_index()
    
    total_paths = []
    patient_ids = []
    series_ids = []
    category = []
    
    for patient_id in range(len(data_to_merge_processed)):
    
        p_id = str(data_to_merge_processed["patient_id"][patient_id]) + "/" + str(data_to_merge_processed["series_id"][patient_id])
        str_imgs_path = path + p_id + '/'
        patient_img_paths = []

        for file in glob(str_imgs_path + '/*'):
            patient_img_paths.append(file)
        
        
        sorted_file_paths = sorted(patient_img_paths, key=extract_number_from_path)
        total_paths.append(sorted_file_paths)
        patient_ids.append(data_to_merge_processed["patient_id"][patient_id])
        series_ids.append(data_to_merge_processed["series_id"][patient_id])
        category.append(data_to_merge_processed["any_injury"][patient_id])
    
    final_data = pd.DataFrame(list(zip(patient_ids, series_ids, total_paths, category)),
               columns =["Patient_id","Series_id", "Patient_paths", "Patient_category"])
    
    return final_data

In [9]:
train_data = pd.read_csv(f"/kaggle/input/rsna-2023-abdominal-trauma-detection/train_series_meta.csv")
cat_data = pd.read_csv(f"/kaggle/input/rsna-2023-abdominal-trauma-detection/train.csv")
path = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/"
cleaned_df = get_data_for_3d_volumes(train_data, cat_data, path=path, number_idx=100)

In [10]:
cleaned_df.head()

,Patient_id,Series_id,Patient_paths,Patient_category
0,25102,50875,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,1
1,31158,28163,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
2,51177,42398,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
3,56046,38794,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0
4,56400,12546,[/kaggle/input/rsna-2023-abdominal-trauma-dete...,0


In [11]:
connection = sqlite3.connect("training_data.db")

In [18]:
cleaned_df["Patient_paths"] = cleaned_df["Patient_paths"].astype(str)

In [19]:
cleaned_df.to_sql(name="base_data", con=connection, if_exists="replace", index=False)

100